https://www.analyticsvidhya.com/blog/2019/01/introduction-time-series-classification/

## Setting up the Problem Statement

We will be working on the ‘Indoor User Movement Prediction‘ problem. In this challenge, multiple motion sensors are placed in different rooms and the goal is to identify whether an individual has moved across rooms, based on the frequency data captured from these motion sensors.

There are four motion sensors (A1, A2, A3, A4) placed across two rooms. Have a look at the below image which illustrates where the sensors are positioned in each room. The setup in these two rooms was created in 3 different pairs of rooms (group1, group2, group3).

A person can move along any of the six pre-defined paths shown in the above image. If a person walks on path 2, 3, 4 or 6, he moves within the room. On the other hand, if a person follows path 1 or path 5, we can say that the person has moved between the rooms.

The sensor reading can be used to identify the position of a person at a given point in time. As the person moves in the room or across rooms, the reading in the sensor changes. This change can be used to identify the path of the person.

Now that the problem statement is clear, it’s time to get down to coding! In the next section, we will look at the dataset for the problem which should help clear up any lingering questions you might have on this statement. You can download the dataset from this link: https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data.

Our dataset comprises of 316 files:

- 314 MovementAAL csv files containing the readings from motion sensors placed in the environment
- A Target csv file that contains the target variable for each MovementAAL file
- One Group Data csv file to identify which MovementAAL file belongs to which setup group
- The Path csv file that contains the path which the object took

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from os import listdir 

import tensorflow as tf 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score

/Users/Jianhua/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/Jianhua/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/Jianhua/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
df1 = pd.read_csv('./MovementAAl/dataset/MovementAAL_RSS_1.csv')
df2 = pd.read_csv('./MovementAAl/dataset/MovementAAL_RSS_2.csv')

In [6]:
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [7]:
df2.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.57143,-0.20,0.71429,0.50
1,-0.76190,-0.48,0.76190,-0.25
2,-0.85714,-0.60,0.85714,0.55
3,-0.76190,-0.40,0.71429,0.60
4,-0.76190,-0.84,0.85714,0.45


In [8]:
df1.shape, df2.shape

((27, 4), (26, 4))

The files contain normalized data from the four sensors – A1, A2, A3, A4. The length of the csv files (number of rows) vary, since the data corresponding to each csv is for a different duration. To simplify things, let us suppose the sensor data is collected every second. The first reading was for a duration of 27 seconds (so 27 rows), while another reading was for 26 seconds (so 26 rows).

We will have to deal with this varying length before we build our model. For now, we will read and store the values from the sensors in a list using the following code block:

In [9]:
path = './MovementAAl/dataset/MovementAAL_RSS_'
sequences = []
for i in range(1, 315):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)
    
targets = pd.read_csv('./MovementAAl/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]

./MovementAAl/dataset/MovementAAL_RSS_1.csv
./MovementAAl/dataset/MovementAAL_RSS_2.csv
./MovementAAl/dataset/MovementAAL_RSS_3.csv
./MovementAAl/dataset/MovementAAL_RSS_4.csv
./MovementAAl/dataset/MovementAAL_RSS_5.csv
./MovementAAl/dataset/MovementAAL_RSS_6.csv
./MovementAAl/dataset/MovementAAL_RSS_7.csv
./MovementAAl/dataset/MovementAAL_RSS_8.csv
./MovementAAl/dataset/MovementAAL_RSS_9.csv
./MovementAAl/dataset/MovementAAL_RSS_10.csv
./MovementAAl/dataset/MovementAAL_RSS_11.csv
./MovementAAl/dataset/MovementAAL_RSS_12.csv
./MovementAAl/dataset/MovementAAL_RSS_13.csv
./MovementAAl/dataset/MovementAAL_RSS_14.csv
./MovementAAl/dataset/MovementAAL_RSS_15.csv
./MovementAAl/dataset/MovementAAL_RSS_16.csv
./MovementAAl/dataset/MovementAAL_RSS_17.csv
./MovementAAl/dataset/MovementAAL_RSS_18.csv
./MovementAAl/dataset/MovementAAL_RSS_19.csv
./MovementAAl/dataset/MovementAAL_RSS_20.csv
./MovementAAl/dataset/MovementAAL_RSS_21.csv
./MovementAAl/dataset/MovementAAL_RSS_22.csv
./MovementAAl/datas

./MovementAAl/dataset/MovementAAL_RSS_230.csv
./MovementAAl/dataset/MovementAAL_RSS_231.csv
./MovementAAl/dataset/MovementAAL_RSS_232.csv
./MovementAAl/dataset/MovementAAL_RSS_233.csv
./MovementAAl/dataset/MovementAAL_RSS_234.csv
./MovementAAl/dataset/MovementAAL_RSS_235.csv
./MovementAAl/dataset/MovementAAL_RSS_236.csv
./MovementAAl/dataset/MovementAAL_RSS_237.csv
./MovementAAl/dataset/MovementAAL_RSS_238.csv
./MovementAAl/dataset/MovementAAL_RSS_239.csv
./MovementAAl/dataset/MovementAAL_RSS_240.csv
./MovementAAl/dataset/MovementAAL_RSS_241.csv
./MovementAAl/dataset/MovementAAL_RSS_242.csv
./MovementAAl/dataset/MovementAAL_RSS_243.csv
./MovementAAl/dataset/MovementAAL_RSS_244.csv
./MovementAAl/dataset/MovementAAL_RSS_245.csv
./MovementAAl/dataset/MovementAAL_RSS_246.csv
./MovementAAl/dataset/MovementAAL_RSS_247.csv
./MovementAAl/dataset/MovementAAL_RSS_248.csv
./MovementAAl/dataset/MovementAAL_RSS_249.csv
./MovementAAl/dataset/MovementAAL_RSS_250.csv
./MovementAAl/dataset/MovementAAL_

In [23]:
targets

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [24]:
groups

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

We now have a list ‘sequences’ that contains the data from the motion sensors and ‘targets’ which holds the labels for the csv files. When we print sequences[0], we get the values of sensors from the first csv file:

In [10]:
sequences[0]

array([[-0.90476 , -0.48    ,  0.28571 ,  0.3     ],
       [-0.57143 , -0.32    ,  0.14286 ,  0.3     ],
       [-0.38095 , -0.28    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.2     , -0.47619 ,  0.35    ],
       [-0.14286 , -0.2     ,  0.14286 , -0.2     ],
       [-0.14286 , -0.2     ,  0.047619,  0.      ],
       [-0.14286 , -0.16    , -0.38095 ,  0.2     ],
       [-0.14286 , -0.04    , -0.61905 , -0.2     ],
       [-0.095238, -0.08    ,  0.14286 , -0.55    ],
       [-0.047619,  0.04    , -0.095238,  0.05    ],
       [-0.19048 , -0.04    ,  0.095238,  0.4     ],
       [-0.095238, -0.04    , -0.14286 ,  0.35    ],
       [-0.33333 , -0.08    , -0.28571 , -0.2     ],
       [-0.2381  ,  0.04    ,  0.14286 ,  0.35    ],
       [ 0.      ,  0.08    ,  0.14286 ,  0.05    ],
       [-0.095238,  0.04    ,  0.095238,  0.1     ],
       [-0.14286 , -0.2     ,  0.14286 ,  0.5     ],
       [-0.19048 ,  0.04    , -0.42857 ,  0.3     ],
       [-0.14286 , -0.08    , -0.2381  ,  0.15

As mentioned previously, the dataset was collected in three different pairs of rooms – hence three groups. This information can be used to divide the dataset into train, test and validation sets. We will load the DatasetGroup csv file now

In [11]:
groups = pd.read_csv('./MovementAAl/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

## Preprocessing Steps

Since the time series data is of varying length, we cannot directly build a model on this dataset. So how can we decide the ideal length of a series? There are multiple ways in which we can deal with it and here are a few ideas (I would love to hear your suggestions in the comment section):

Pad the shorter sequences with zeros to make the length of all the series equal. In this case, we will be feeding incorrect data to the model
Find the maximum length of the series and pad the sequence with the data in the last row
Identify the minimum length of the series in the dataset and truncate all the other series to that length. However, this will result in a huge loss of data
Take the mean of all the lengths, truncate the longer series, and pad the series which are shorter than the mean length

Let’s find out the minimum, maximum and mean length:

In [12]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
    
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

Most of the files have lengths between 40 to 60. Just 3 files are coming up with a length more than 100. Thus, taking the minimum or maximum length does not make much sense. The 90th quartile comes out to be 60, which is taken as the length of sequence for the data. Let’s code it out:

In [13]:
# padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)

final_seq = np.stack(new_seq)

# truncate the sequence to length 60
seq_len = 60 
final_seq = sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [18]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i] == 2)]
validation = [final_seq[i] for i in range(len(groups)) if (groups[i] == 1)]
test = [final_seq[i] for i in range(len(groups)) if groups[i] == 3]
train_target = [targets[i] for i in range(len(groups)) if (groups[i] == 2)]
validation_target = [targets[i] for i in range(len(groups)) if (groups[i] == 1)]
test_target = [targets[i] for i in range(len(groups)) if (groups[i] == 3)]
train = np.array(train)
validation = np.array(validation)
test = np.array(test)
train_target = np.array(train_target)
train_target = (train_target + 1) / 2
validation_target = np.array(validation_target)
validation_target = (validation_target + 1) / 2
test_target = np.array(test_target)
test_target = (test_target + 1) / 2

## Building a Time Series Classification model

We have prepared the data to be used for an LSTM (Long Short Term Memory) model. We dealt with the variable length sequence and created the train, validation and test sets. Let’s build a single layer LSTM network.

In [19]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

W0717 10:32:44.759997 4472313280 deprecation_wrapper.py:119] From /Users/Jianhua/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 10:32:45.072867 4472313280 deprecation_wrapper.py:119] From /Users/Jianhua/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 10:32:45.179292 4472313280 deprecation_wrapper.py:119] From /Users/Jianhua/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               267264    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [22]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation, validation_target))

# loading the model and checking accuracy on the test data 
model = load_model('best_model.pkl')
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

Train on 106 samples, validate on 104 samples
Epoch 1/200
106/106 [==============================] - 2s 19ms/step - loss: 0.5083 - acc: 0.8396 - val_loss: 1.6572 - val_acc: 0.5962

Epoch 00001: val_acc improved from -inf to 0.59615, saving model to best_model.pkl
Epoch 2/200
106/106 [==============================] - 1s 5ms/step - loss: 0.1780 - acc: 0.9717 - val_loss: 1.7219 - val_acc: 0.5962

Epoch 00002: val_acc did not improve from 0.59615
Epoch 3/200
106/106 [==============================] - 1s 5ms/step - loss: 0.1446 - acc: 0.9623 - val_loss: 1.7110 - val_acc: 0.6154

Epoch 00003: val_acc improved from 0.59615 to 0.61538, saving model to best_model.pkl
Epoch 4/200
106/106 [==============================] - 1s 8ms/step - loss: 0.1683 - acc: 0.9623 - val_loss: 1.5925 - val_acc: 0.6058

Epoch 00004: val_acc did not improve from 0.61538
Epoch 5/200
106/106 [==============================] - 1s 6ms/step - loss: 0.1098 - acc: 0.9717 - val_loss: 1.5153 - val_acc: 0.5769

Epoch 00005: v


Epoch 00044: val_acc did not improve from 0.67308
Epoch 45/200
106/106 [==============================] - 1s 7ms/step - loss: 0.0735 - acc: 0.9906 - val_loss: 1.9522 - val_acc: 0.5962

Epoch 00045: val_acc did not improve from 0.67308
Epoch 46/200
106/106 [==============================] - 1s 10ms/step - loss: 0.0697 - acc: 0.9906 - val_loss: 1.9867 - val_acc: 0.5962

Epoch 00046: val_acc did not improve from 0.67308
Epoch 47/200
106/106 [==============================] - 1s 7ms/step - loss: 0.0659 - acc: 0.9906 - val_loss: 2.0211 - val_acc: 0.5865

Epoch 00047: val_acc did not improve from 0.67308
Epoch 48/200
106/106 [==============================] - 1s 7ms/step - loss: 0.0633 - acc: 0.9906 - val_loss: 2.0436 - val_acc: 0.5865

Epoch 00048: val_acc did not improve from 0.67308
Epoch 49/200
106/106 [==============================] - 1s 8ms/step - loss: 0.0595 - acc: 1.0000 - val_loss: 2.0609 - val_acc: 0.5865

Epoch 00049: val_acc did not improve from 0.67308
Epoch 50/200
106/106 [=

106/106 [==============================] - 1s 8ms/step - loss: 0.2827 - acc: 0.9057 - val_loss: 1.9304 - val_acc: 0.5577

Epoch 00089: val_acc did not improve from 0.67308
Epoch 90/200
106/106 [==============================] - 1s 6ms/step - loss: 0.3010 - acc: 0.9057 - val_loss: 1.9487 - val_acc: 0.5673

Epoch 00090: val_acc did not improve from 0.67308
Epoch 91/200
106/106 [==============================] - 1s 8ms/step - loss: 0.2681 - acc: 0.9245 - val_loss: 1.8768 - val_acc: 0.5865

Epoch 00091: val_acc did not improve from 0.67308
Epoch 92/200
106/106 [==============================] - 1s 6ms/step - loss: 0.2328 - acc: 0.9340 - val_loss: 1.8399 - val_acc: 0.5769

Epoch 00092: val_acc did not improve from 0.67308
Epoch 93/200
106/106 [==============================] - 1s 5ms/step - loss: 0.2027 - acc: 0.9340 - val_loss: 1.8559 - val_acc: 0.5865

Epoch 00093: val_acc did not improve from 0.67308
Epoch 94/200
106/106 [==============================] - 0s 4ms/step - loss: 0.2049 - acc

106/106 [==============================] - 1s 6ms/step - loss: 0.3881 - acc: 0.8491 - val_loss: 0.5477 - val_acc: 0.7404

Epoch 00132: val_acc did not improve from 0.76923
Epoch 133/200
106/106 [==============================] - 0s 5ms/step - loss: 0.6319 - acc: 0.7642 - val_loss: 0.8921 - val_acc: 0.5577

Epoch 00133: val_acc did not improve from 0.76923
Epoch 134/200
106/106 [==============================] - 1s 5ms/step - loss: 0.2820 - acc: 0.8962 - val_loss: 0.9086 - val_acc: 0.5481

Epoch 00134: val_acc did not improve from 0.76923
Epoch 135/200
106/106 [==============================] - 0s 4ms/step - loss: 0.5348 - acc: 0.7075 - val_loss: 0.8752 - val_acc: 0.5673

Epoch 00135: val_acc did not improve from 0.76923
Epoch 136/200
106/106 [==============================] - 0s 4ms/step - loss: 0.4280 - acc: 0.8208 - val_loss: 0.7577 - val_acc: 0.6346

Epoch 00136: val_acc did not improve from 0.76923
Epoch 137/200
106/106 [==============================] - 0s 4ms/step - loss: 0.7373 


Epoch 00176: val_acc did not improve from 0.76923
Epoch 177/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3682 - acc: 0.8113 - val_loss: 0.8741 - val_acc: 0.5000

Epoch 00177: val_acc did not improve from 0.76923
Epoch 178/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3631 - acc: 0.8208 - val_loss: 0.8662 - val_acc: 0.5000

Epoch 00178: val_acc did not improve from 0.76923
Epoch 179/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3966 - acc: 0.7925 - val_loss: 0.8771 - val_acc: 0.5000

Epoch 00179: val_acc did not improve from 0.76923
Epoch 180/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3529 - acc: 0.8113 - val_loss: 0.8868 - val_acc: 0.5000

Epoch 00180: val_acc did not improve from 0.76923
Epoch 181/200
106/106 [==============================] - 0s 4ms/step - loss: 0.4136 - acc: 0.8019 - val_loss: 0.8782 - val_acc: 0.5000

Epoch 00181: val_acc did not improve from 0.76923
Epoch 182/200
106/1

0.7692307692307693